In [ ]:
!pip install nest_asyncio llama-index-core llama-index-llms-openai llama-index-utils-workflow llama-index-readers-file llama-index-embeddings-openai markdown

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Attempting u

In [ ]:
# Version 1.0
import os
import asyncio
import nest_asyncio
from asyncio import get_event_loop
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.workflow import step, Context, Workflow, Event, StartEvent, StopEvent
from llama_index.llms.openai import OpenAI
from llama_index.utils.workflow import draw_all_possible_flows
import subprocess


os.makedirs("data", exist_ok=True)

# to download from URL
#subprocess.run(["wget", "url", "-O", "data/url.pdf"])

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = "Your_api_key_here"

# Define event classes
class StrategyQueryEvent(Event):
    query: str
    company_id: str

class VagueStrategyEvent(Event):
    query: str
    company_id: str

class CompanyHistoryEvent(Event):
    query: str
    company_id: str

class MarketAnalysisEvent(Event):
    query: str
    company_id: str

class BlueOceanEvent(Event):
    query: str
    company_id: str

class ResponseEvent(Event):
    query: str
    source: str
    response: str

# Define the workflow
class BusinessStrategyWorkflow(Workflow):
    def load_or_create_index(self, directory_path, persist_dir):
        full_directory_path = directory_path
        full_persist_dir = os.path.join("data", persist_dir)
        if os.path.exists(full_persist_dir):
            print(f"Loading existing index from {full_persist_dir}...")
            storage_context = StorageContext.from_defaults(persist_dir=full_persist_dir)
            index = load_index_from_storage(storage_context)
        else:
            print(f"Creating new index from {full_directory_path}...")
            documents = SimpleDirectoryReader(full_directory_path).load_data()
            index = VectorStoreIndex.from_documents(documents)
            index.storage_context.persist(persist_dir=full_persist_dir)
        return index

    @step
    async def analyze_strategy_query(self, ctx: Context, ev: StartEvent | StrategyQueryEvent) -> VagueStrategyEvent | CompanyHistoryEvent | MarketAnalysisEvent | BlueOceanEvent:
        if not hasattr(ctx.data, "llm"):
            ctx.data["llm"] = OpenAI(model="gpt-4o-mini", temperature=0.1)
            ctx.data["company_index"] = self.load_or_create_index("data", "company_history")
            ctx.data["market_index"] = self.load_or_create_index("data", "market_analysis")

        query_analysis = ctx.data["llm"].complete(
            f"Analyze this strategy query: '{ev.query}'. Determine if it is clear or vague. If clear, return 'clear'. If vague, return 'vague'."
        ).text
        if query_analysis == "vague":
            return VagueStrategyEvent(query=ev.query, company_id=ev.company_id)
        else:
            return CompanyHistoryEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def refine_strategy_query(self, ctx: Context, ev: VagueStrategyEvent) -> StrategyQueryEvent:
        # Refine the vague strategy query
        refined_query = ctx.data["llm"].complete(
            f"The query '{ev.query}' is too vague for a detailed strategy analysis. Please provide a more specific query."
        ).text
        return StrategyQueryEvent(query=str(refined_query), company_id=ev.company_id)

    @step
    async def company_history_analysis(self, ctx: Context, ev: CompanyHistoryEvent) -> MarketAnalysisEvent:
        # Analyze Innowave's company history
        company_history_analysis = ctx.data["company_index"].as_query_engine().query(
            f"Analyze Innowave's company history and its implications for the strategy related to '{ev.query}'."
        )
        ctx.data["company_history_response"] = company_history_analysis
        return MarketAnalysisEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def market_analysis(self, ctx: Context, ev: MarketAnalysisEvent) -> BlueOceanEvent:
        # Analyze market conditions
        market_analysis = ctx.data["market_index"].as_query_engine().query(
            f"Analyze the market conditions and competitive landscape for Innowave's strategy in '{ev.query}'."
        )
        ctx.data["market_analysis_response"] = market_analysis
        return BlueOceanEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def four_actions_analysis(self, ctx: Context, ev: BlueOceanEvent) -> ResponseEvent:
        # Analyze Innowave's current strategy and identify factors to eliminate, reduce, raise, and create
        blue_ocean_analysis = ctx.data["llm"].complete(
            f"Analyze Innowave's current strategy and apply the Four Actions Framework (eliminate, reduce, raise, create) to identify Blue Ocean opportunities in the '{ev.query}' domain."
        ).text
        return ResponseEvent(query=ev.query, source="Four Actions Analysis", response=blue_ocean_analysis)

    @step
    async def strategy_canvas(self, ctx: Context, ev: BlueOceanEvent) -> ResponseEvent:
        # Generate a strategy canvas comparing Innowave to its competitors
        canvas_analysis = ctx.data["llm"].complete(
            f"Generate a strategy canvas comparing Innowave's competitive factors with competitors in the market. Highlight areas for value innovation."
        ).text
        return ResponseEvent(query=ev.query, source="Strategy Canvas", response=canvas_analysis)

    @step
    async def generate_strategy_recommendations(self, ctx: Context, ev: ResponseEvent) -> StopEvent | None:
        ready = ctx.collect_events(ev, [ResponseEvent] * 3)
        if ready is None:
            print("Not enough responses collected, returning None")
            return None

        print("Generating strategy recommendations...")
        try:
            #print("Generating strategy recommendations...")
            company_history = ctx.data.get("company_history_response", "No company history available")
            market_analysis = ctx.data.get("market_analysis_response", "No market analysis available")
            blue_ocean_insights = ready[0].response if ready else "No Blue Ocean insights available"
            strategy_recommendations = ctx.data["llm"].complete(
                f"Create a strategic recommendation based on: "
                f"\n1. Company History: {ctx.data['company_history_response']}"
                f"\n2. Market Analysis: {ctx.data['market_analysis_response']}"
               f"\n3. Blue Ocean Insights: {ready[0].response}"
            ).text
            print("Strategy recommendations generated successfully.")
        except Exception as e:
            print(f"Error during strategy generation: {e}")
            return None
        return StopEvent(result=str(strategy_recommendations))

# Draw the workflow diagram
draw_all_possible_flows(BusinessStrategyWorkflow, filename="business_strategy_workflow.html")

# Set up asyncio for Jupyter
nest_asyncio.apply()

# Run the workflow
async def run_workflow():
    workflow = BusinessStrategyWorkflow(timeout=180, verbose=True)
    result = await workflow.run(query="Identify untapped market segments that InnoWave Inc. can explore to differentiate itself from competitors.", company_id="Innowave")
    return result

# Execute the workflow
result = get_event_loop().run_until_complete(run_workflow())
print(result)

# Display the result in markdown format
from IPython.display import display, HTML
import markdown

display(HTML(markdown.markdown(result)))


business_strategy_workflow.html
Running step analyze_strategy_query
Creating new index from data...


Creating new index from data...


Step analyze_strategy_query produced event CompanyHistoryEvent
Running step company_history_analysis
Step company_history_analysis produced event MarketAnalysisEvent
Running step market_analysis
Step market_analysis produced event BlueOceanEvent
Running step strategy_canvas
Step strategy_canvas produced event ResponseEvent
Running step four_actions_analysis
Step four_actions_analysis produced event ResponseEvent
Running step generate_strategy_recommendations
Not enough responses collected, returning None
Step generate_strategy_recommendations produced no event
Running step generate_strategy_recommendations
Not enough responses collected, returning None
Step generate_strategy_recommendations produced no event


WorkflowTimeoutError: Operation timed out after 180 seconds

In [ ]:
# this block increases the time out to 600 seconds
# Run the workflow
async def run_workflow():
    workflow = BusinessStrategyWorkflow(timeout=600, verbose=True)
    result = await workflow.run(query="Identify untapped market segments that InnoWave Inc. can explore to differentiate itself from competitors.", company_id="Innowave")
    return result

# Execute the workflow
result = get_event_loop().run_until_complete(run_workflow())
print(result)

# Display the result in markdown format
from IPython.display import display, HTML
import markdown

display(HTML(markdown.markdown(result)))

Running step analyze_strategy_query
Loading existing index from data/company_history...
Loading existing index from data/market_analysis...
Step analyze_strategy_query produced event CompanyHistoryEvent
Running step company_history_analysis
Step company_history_analysis produced event MarketAnalysisEvent
Running step market_analysis
Step market_analysis produced event BlueOceanEvent
Running step strategy_canvas
Step strategy_canvas produced event ResponseEvent
Running step four_actions_analysis
Step four_actions_analysis produced event ResponseEvent
Running step generate_strategy_recommendations
Not enough responses collected, returning None
Step generate_strategy_recommendations produced no event
Running step generate_strategy_recommendations
Not enough responses collected, returning None
Step generate_strategy_recommendations produced no event


WorkflowTimeoutError: Operation timed out after 600 seconds

In [ ]:
# Sctratch code used to copy paste
## @step
# async def generate_strategy_recommendations(self, ctx: Context, ev: ResponseEvent) -> StopEvent | None:
#     ready = ctx.collect_events(ev, [ResponseEvent] * 3)
#     if ready is None:
#         print("Not enough responses collected, returning None")
#         return None

#     try:
#         print("Generating strategy recommendations...")
#         strategy_recommendations = ctx.data["llm"].complete(
#             f"Create a strategic recommendation based on: "
#             f"\n1. Company History: {ctx.data['company_history_response']}"
#             f"\n2. Market Analysis: {ctx.data['market_analysis_response']}"
#             f"\n3. Blue Ocean Insights: {ready[0].response}"
#         ).text
#         print("Strategy recommendations generated successfully.")
#     except Exception as e:
#         print(f"Error during strategy generation: {e}")
#         return None

#     return StopEvent(result=str(strategy_recommendations))

In [ ]:
#Anotehr verison
import os
import asyncio
import nest_asyncio
from asyncio import get_event_loop
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.workflow import step, Context, Workflow, Event, StartEvent, StopEvent
from llama_index.llms.openai import OpenAI
from llama_index.utils.workflow import draw_all_possible_flows
import subprocess


os.makedirs("data", exist_ok=True)

# url
#subprocess.run(["wget", "url", "-O", "data/url.pdf"])

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = "Your_api_key_here"

# Define event classes
class StrategyQueryEvent(Event):
    query: str
    company_id: str

class VagueStrategyEvent(Event):
    query: str
    company_id: str

class CompanyHistoryEvent(Event):
    query: str
    company_id: str

class MarketAnalysisEvent(Event):
    query: str
    company_id: str

class BlueOceanEvent(Event):
    query: str
    company_id: str

class ResponseEvent(Event):
    query: str
    source: str
    response: str

# Define the workflow
class BusinessStrategyWorkflow(Workflow):
    def load_or_create_index(self, directory_path, persist_dir):
        full_directory_path = directory_path
        full_persist_dir = os.path.join("data", persist_dir)
        if os.path.exists(full_persist_dir):
            print(f"Loading existing index from {full_persist_dir}...")
            storage_context = StorageContext.from_defaults(persist_dir=full_persist_dir)
            index = load_index_from_storage(storage_context)
        else:
            print(f"Creating new index from {full_directory_path}...")
            documents = SimpleDirectoryReader(full_directory_path).load_data()
            index = VectorStoreIndex.from_documents(documents)
            index.storage_context.persist(persist_dir=full_persist_dir)
        return index

    @step
    async def analyze_strategy_query(self, ctx: Context, ev: StartEvent | StrategyQueryEvent) -> VagueStrategyEvent | CompanyHistoryEvent | MarketAnalysisEvent | BlueOceanEvent:
        if not hasattr(ctx.data, "llm"):
            ctx.data["llm"] = OpenAI(model="gpt-4o-mini", temperature=0.1)
            ctx.data["company_index"] = self.load_or_create_index("data", "company_history")
            ctx.data["market_index"] = self.load_or_create_index("data", "market_analysis")

        query_analysis = ctx.data["llm"].complete(
            f"Analyze this strategy query: '{ev.query}'. Determine if it is clear or vague. If clear, return 'clear'. If vague, return 'vague'."
        ).text
        if query_analysis == "vague":
            return VagueStrategyEvent(query=ev.query, company_id=ev.company_id)
        else:
            return CompanyHistoryEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def refine_strategy_query(self, ctx: Context, ev: VagueStrategyEvent) -> StrategyQueryEvent:
        # Refine the vague strategy query
        refined_query = ctx.data["llm"].complete(
            f"The query '{ev.query}' is too vague for a detailed strategy analysis. Please provide a more specific query."
        ).text
        return StrategyQueryEvent(query=str(refined_query), company_id=ev.company_id)

    @step
    async def company_history_analysis(self, ctx: Context, ev: CompanyHistoryEvent) -> MarketAnalysisEvent:
        # Analyze Innowave's company history
        company_history_analysis = ctx.data["company_index"].as_query_engine().query(
            f"Analyze Innowave's company history and its implications for the strategy related to '{ev.query}'."
        )
        if company_history_analysis:
            print("Company History Analysis produced a response.")
        else:
            print("Company History Analysis did not produce a response.")
        ctx.data["company_history_response"] = company_history_analysis
        return MarketAnalysisEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def market_analysis(self, ctx: Context, ev: MarketAnalysisEvent) -> BlueOceanEvent:
        # Analyze market conditions
        market_analysis = ctx.data["market_index"].as_query_engine().query(
            f"Analyze the market conditions and competitive landscape for Innowave's strategy in '{ev.query}'."
        )
        if market_analysis:
            print("Market Analysis produced a response.")
        else:
            print("Market Analysis did not produce a response.")
        ctx.data["market_analysis_response"] = market_analysis
        return BlueOceanEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def strategy_canvas(self, ctx: Context, ev: BlueOceanEvent) -> ResponseEvent:
        # Generate a strategy canvas comparing Innowave to its competitors
        canvas_analysis = ctx.data["llm"].complete(
            f"Generate a strategy canvas comparing Innowave's competitive factors with competitors in the market. Highlight areas for value innovation."
        ).text
        return ResponseEvent(query=ev.query, source="Strategy Canvas", response=canvas_analysis)

    @step
    async def four_actions_analysis(self, ctx: Context, ev: BlueOceanEvent) -> ResponseEvent:
        # Analyze Innowave's current strategy and identify factors to eliminate, reduce, raise, and create
        blue_ocean_analysis = ctx.data["llm"].complete(
            f"Analyze Innowave's current strategy and apply the Four Actions Framework (eliminate, reduce, raise, create) to identify Blue Ocean opportunities in the '{ev.query}' domain."
        ).text
        return ResponseEvent(query=ev.query, source="Four Actions Analysis", response=blue_ocean_analysis)

    @step
    async def generate_strategy_recommendations(self, ctx: Context, ev: ResponseEvent) -> StopEvent | None:
        ready = ctx.collect_events(ev, [ResponseEvent] * 2)
        print(f"Collected events: {ready}")

        if not ready or len(ready) < 2:
           print("Not enough responses collected, stopping execution.")
           return StopEvent(result="Unable to generate recommendations due to insufficient data.")

        try:
           company_history = ctx.data.get("company_history_response", "No company history available")
           market_analysis = ctx.data.get("market_analysis_response", "No market analysis available")
           strategy_canvas_response = None
           four_actions_response = None

           for response in ready:
               if response.source == "Strategy Canvas":
                   strategy_canvas_response = response.response
               elif response.source == "Four Actions Analysis":
                   four_actions_response = response.response

           strategy_recommendations = ctx.data["llm"].complete(
               f"Create a strategic recommendation based on: "
               f"\n1. Company History: {company_history}"
               f"\n2. Market Analysis: {market_analysis}"
               f"\n3. Strategy Canvas: {strategy_canvas_response}"
               f"\n4. Four Actions Analysis: {four_actions_response}"
           ).text
           print("Strategy recommendations generated successfully.")
           return StopEvent(result=str(strategy_recommendations))
        except Exception as e:
           print(f"Error during strategy generation: {e}")
           return StopEvent(result="Error in generating strategy recommendations.")

# Draw the workflow diagram
draw_all_possible_flows(BusinessStrategyWorkflow, filename="business_strategy_workflow.html")

# Set up asyncio for Jupyter
nest_asyncio.apply()

# Run the workflow
async def run_workflow():
    workflow = BusinessStrategyWorkflow(timeout=600, verbose=True)
    result = await workflow.run(query="Identify untapped market segments that InnoWave Inc. can explore to differentiate itself from competitors.", company_id="innowave")
    return result

# Execute the workflow
result = get_event_loop().run_until_complete(run_workflow())
print(result)

# Display the result in markdown format
from IPython.display import display, HTML
import markdown

display(HTML(markdown.markdown(result)))


business_strategy_workflow.html
Running step analyze_strategy_query
Loading existing index from data/company_history...
Loading existing index from data/market_analysis...
Step analyze_strategy_query produced event CompanyHistoryEvent
Running step company_history_analysis
Company History Analysis produced a response.
Step company_history_analysis produced event MarketAnalysisEvent
Running step market_analysis
Market Analysis produced a response.
Step market_analysis produced event BlueOceanEvent
Running step strategy_canvas
Step strategy_canvas produced event ResponseEvent
Running step four_actions_analysis
Step four_actions_analysis produced event ResponseEvent
Running step generate_strategy_recommendations
Collected events: None
Not enough responses collected, stopping execution.
Step generate_strategy_recommendations produced event StopEvent
Running step generate_strategy_recommendations
Collected events: [ResponseEvent(query='Identify untapped market segments that InnoWave Inc. can

In [ ]:
# Complete Code for Partial Generation Strategy
import os
import asyncio
import nest_asyncio
from asyncio import get_event_loop
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.workflow import step, Context, Workflow, Event, StartEvent, StopEvent
from llama_index.llms.openai import OpenAI
from llama_index.utils.workflow import draw_all_possible_flows
import subprocess


os.makedirs("data", exist_ok=True)



# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = "your_api_key_here"

# Define event classes
class StrategyQueryEvent(Event):
    query: str
    company_id: str

class VagueStrategyEvent(Event):
    query: str
    company_id: str

class CompanyHistoryEvent(Event):
    query: str
    company_id: str

class MarketAnalysisEvent(Event):
    query: str
    company_id: str

class BlueOceanEvent(Event):
    query: str
    company_id: str

class ResponseEvent(Event):
    query: str
    source: str
    response: str

# Define the workflow
class BusinessStrategyWorkflow(Workflow):
    def load_or_create_index(self, directory_path, persist_dir):
        full_directory_path = directory_path
        full_persist_dir = os.path.join("data", persist_dir)
        if os.path.exists(full_persist_dir):
            print(f"Loading existing index from {full_persist_dir}...")
            storage_context = StorageContext.from_defaults(persist_dir=full_persist_dir)
            index = load_index_from_storage(storage_context)
        else:
            print(f"Creating new index from {full_directory_path}...")
            documents = SimpleDirectoryReader(full_directory_path).load_data()
            index = VectorStoreIndex.from_documents(documents)
            index.storage_context.persist(persist_dir=full_persist_dir)
        return index

    @step
    async def analyze_strategy_query(self, ctx: Context, ev: StartEvent | StrategyQueryEvent) -> VagueStrategyEvent | CompanyHistoryEvent | MarketAnalysisEvent | BlueOceanEvent:
        if not hasattr(ctx.data, "llm"):
            ctx.data["llm"] = OpenAI(model="gpt-4o-mini", temperature=0.1)
            ctx.data["company_index"] = self.load_or_create_index("data", "company_history")
            ctx.data["market_index"] = self.load_or_create_index("data", "market_analysis")

        query_analysis = ctx.data["llm"].complete(
            f"Analyze this strategy query: '{ev.query}'. Determine if it is clear or vague. If clear, return 'clear'. If vague, return 'vague'."
        ).text
        if query_analysis == "vague":
            return VagueStrategyEvent(query=ev.query, company_id=ev.company_id)
        else:
            return CompanyHistoryEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def refine_strategy_query(self, ctx: Context, ev: VagueStrategyEvent) -> StrategyQueryEvent:
        # Refine the vague strategy query
        refined_query = ctx.data["llm"].complete(
            f"The query '{ev.query}' is too vague for a detailed strategy analysis. Please provide a more specific query."
        ).text
        return StrategyQueryEvent(query=str(refined_query), company_id=ev.company_id)

    @step
    async def company_history_analysis(self, ctx: Context, ev: CompanyHistoryEvent) -> MarketAnalysisEvent:
        # Analyze Innowave's company history
        company_history_analysis = ctx.data["company_index"].as_query_engine().query(
            f"Analyze Innowave's company history and its implications for the strategy related to '{ev.query}'."
        )
        if company_history_analysis:
            print("Company History Analysis produced a response.")
        else:
            print("Company History Analysis did not produce a response.")
        ctx.data["company_history_response"] = company_history_analysis
        return MarketAnalysisEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def market_analysis(self, ctx: Context, ev: MarketAnalysisEvent) -> BlueOceanEvent:
        # Analyze market conditions
        market_analysis = ctx.data["market_index"].as_query_engine().query(
            f"Analyze the market conditions and competitive landscape for Innowave's strategy in '{ev.query}'."
        )
        if market_analysis:
            print("Market Analysis produced a response.")
        else:
            print("Market Analysis did not produce a response.")
        ctx.data["market_analysis_response"] = market_analysis
        return BlueOceanEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def strategy_canvas(self, ctx: Context, ev: BlueOceanEvent) -> ResponseEvent:
        # Generate a strategy canvas comparing Innowave to its competitors
        canvas_analysis = ctx.data["llm"].complete(
            f"Generate a strategy canvas comparing Innowave's competitive factors with competitors in the market. Highlight areas for value innovation."
        ).text
        return ResponseEvent(query=ev.query, source="Strategy Canvas", response=canvas_analysis)

    @step
    async def four_actions_analysis(self, ctx: Context, ev: BlueOceanEvent) -> ResponseEvent:
        # Analyze Innowave's current strategy and identify factors to eliminate, reduce, raise, and create
        blue_ocean_analysis = ctx.data["llm"].complete(
            f"Analyze Innowave's current strategy and apply the Four Actions Framework (eliminate, reduce, raise, create) to identify Blue Ocean opportunities in the '{ev.query}' domain."
        ).text
        return ResponseEvent(query=ev.query, source="Four Actions Analysis", response=blue_ocean_analysis)

    @step
    async def generate_strategy_recommendations(self, ctx: Context, ev: ResponseEvent) -> StopEvent | None:
        ready = ctx.collect_events(ev, [ResponseEvent] * 2)
        print(f"Collected events: {ready}")

        if not ready or len(ready) < 2:
           print("Not enough responses collected, generating partial recommendations.")

           company_history = ctx.data.get("company_history_response", "No company history available")
           market_analysis = ctx.data.get("market_analysis_response", "No market analysis available")

           partial_recommendations = []

           # Generate based on company history
           partial_recommendations.append(
               ctx.data["llm"].complete(
                   f"Generate strategic insights based on company history: {company_history}"
               ).text
           )

           # Generate based on market analysis
           partial_recommendations.append(
               ctx.data["llm"].complete(
                   f"Generate strategic insights based on market analysis: {market_analysis}"
               ).text
           )

           # Combine partial recommendations
           final_recommendation = "\n\n".join(partial_recommendations)

           return StopEvent(result=final_recommendation)

        try:
           company_history = ctx.data.get("company_history_response", "No company history available")
           market_analysis = ctx.data.get("market_analysis_response", "No market analysis available")
           strategy_canvas_response = None
           four_actions_response = None

           for response in ready:
               if response.source == "Strategy Canvas":
                   strategy_canvas_response = response.response
               elif response.source == "Four Actions Analysis":
                   four_actions_response = response.response

           # Generate recommendations with all available data
           strategy_recommendations = ctx.data["llm"].complete(
               f"Create a strategic recommendation based on: "
               f"\n1. Company History: {company_history}"
               f"\n2. Market Analysis: {market_analysis}"
               f"\n3. Strategy Canvas: {strategy_canvas_response or 'No Strategy Canvas available'}"
               f"\n4. Four Actions Analysis: {four_actions_response or 'No Four Actions Analysis available'}"
           ).text
           print("Strategy recommendations generated successfully.")
           return StopEvent(result=str(strategy_recommendations))

        except Exception as e:
           print(f"Error during strategy generation: {e}")
           return StopEvent(result="Error in generating strategy recommendations.")

# Draw the workflow diagram
draw_all_possible_flows(BusinessStrategyWorkflow, filename="business_strategy_workflow.html")

# Set up asyncio for Jupyter
nest_asyncio.apply()

# Run the workflow
async def run_workflow():
    workflow = BusinessStrategyWorkflow(timeout=600, verbose=True)
    result = await workflow.run(query="Identify untapped market segments that InnoWave Inc. can explore to differentiate itself from competitors.", company_id="Innowave")
    return result

# Execute the workflow
result = get_event_loop().run_until_complete(run_workflow())
print(result)

# Create a new workflow instance and draw its execution
workflow = BusinessStrategyWorkflow(timeout=600, verbose=False)
# Execute the workflow to generate execution history
_ = get_event_loop().run_until_complete(workflow.run(query="Identify untapped market segments that InnoWave Inc. can explore to differentiate itself from competitors.", company_id="Innowave"))
draw_most_recent_execution(workflow, filename="1business_strategy_flow_recent.html")

# Display the result in markdown format
from IPython.display import display, HTML
import markdown

display(HTML(markdown.markdown(result)))


business_strategy_workflow.html
Running step analyze_strategy_query
Loading existing index from data/company_history...
Loading existing index from data/market_analysis...
Step analyze_strategy_query produced event CompanyHistoryEvent
Running step company_history_analysis
Company History Analysis produced a response.
Step company_history_analysis produced event MarketAnalysisEvent
Running step market_analysis
Market Analysis produced a response.
Step market_analysis produced event BlueOceanEvent
Running step strategy_canvas
Step strategy_canvas produced event ResponseEvent
Running step four_actions_analysis
Step four_actions_analysis produced event ResponseEvent
Running step generate_strategy_recommendations
Collected events: None
Not enough responses collected, generating partial recommendations.
Step generate_strategy_recommendations produced event StopEvent
Running step generate_strategy_recommendations
Collected events: [ResponseEvent(query='Identify untapped market segments that I

In [ ]:
# code for Fallback Strategy Using Available Data
import os
import asyncio
import nest_asyncio
from asyncio import get_event_loop
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.workflow import step, Context, Workflow, Event, StartEvent, StopEvent
from llama_index.llms.openai import OpenAI
from llama_index.utils.workflow import draw_all_possible_flows
import subprocess


os.makedirs("data", exist_ok=True)



# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = "Your_api_key_here"

# Define event classes
class StrategyQueryEvent(Event):
    query: str
    company_id: str

class VagueStrategyEvent(Event):
    query: str
    company_id: str

class CompanyHistoryEvent(Event):
    query: str
    company_id: str

class MarketAnalysisEvent(Event):
    query: str
    company_id: str

class BlueOceanEvent(Event):
    query: str
    company_id: str

class ResponseEvent(Event):
    query: str
    source: str
    response: str

# Define the workflow
class BusinessStrategyWorkflow(Workflow):
    def load_or_create_index(self, directory_path, persist_dir):
        full_directory_path = directory_path
        full_persist_dir = os.path.join("data", persist_dir)
        if os.path.exists(full_persist_dir):
            print(f"Loading existing index from {full_persist_dir}...")
            storage_context = StorageContext.from_defaults(persist_dir=full_persist_dir)
            index = load_index_from_storage(storage_context)
        else:
            print(f"Creating new index from {full_directory_path}...")
            documents = SimpleDirectoryReader(full_directory_path).load_data()
            index = VectorStoreIndex.from_documents(documents)
            index.storage_context.persist(persist_dir=full_persist_dir)
        return index

    @step
    async def analyze_strategy_query(self, ctx: Context, ev: StartEvent | StrategyQueryEvent) -> VagueStrategyEvent | CompanyHistoryEvent | MarketAnalysisEvent | BlueOceanEvent:
        if not hasattr(ctx.data, "llm"):
            ctx.data["llm"] = OpenAI(model="gpt-4o-mini", temperature=0.1)
            ctx.data["company_index"] = self.load_or_create_index("data", "company_history")
            ctx.data["market_index"] = self.load_or_create_index("data", "market_analysis")

        query_analysis = ctx.data["llm"].complete(
            f"Analyze this strategy query: '{ev.query}'. Determine if it is clear or vague. If clear, return 'clear'. If vague, return 'vague'."
        ).text
        if query_analysis == "vague":
            return VagueStrategyEvent(query=ev.query, company_id=ev.company_id)
        else:
            return CompanyHistoryEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def refine_strategy_query(self, ctx: Context, ev: VagueStrategyEvent) -> StrategyQueryEvent:
        # Refine the vague strategy query
        refined_query = ctx.data["llm"].complete(
            f"The query '{ev.query}' is too vague for a detailed strategy analysis. Please provide a more specific query."
        ).text
        return StrategyQueryEvent(query=str(refined_query), company_id=ev.company_id)

    @step
    async def company_history_analysis(self, ctx: Context, ev: CompanyHistoryEvent) -> MarketAnalysisEvent:
        # Analyze Innowave's company history
        company_history_analysis = ctx.data["company_index"].as_query_engine().query(
            f"Analyze Innowave's company history and its implications for the strategy related to '{ev.query}'."
        )
        if company_history_analysis:
            print("Company History Analysis produced a response.")
        else:
            print("Company History Analysis did not produce a response.")
        ctx.data["company_history_response"] = company_history_analysis
        return MarketAnalysisEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def market_analysis(self, ctx: Context, ev: MarketAnalysisEvent) -> BlueOceanEvent:
        # Analyze market conditions
        market_analysis = ctx.data["market_index"].as_query_engine().query(
            f"Analyze the market conditions and competitive landscape for Innowave's strategy in '{ev.query}'."
        )
        if market_analysis:
            print("Market Analysis produced a response.")
        else:
            print("Market Analysis did not produce a response.")
        ctx.data["market_analysis_response"] = market_analysis
        return BlueOceanEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def strategy_canvas(self, ctx: Context, ev: BlueOceanEvent) -> ResponseEvent:
        # Generate a strategy canvas comparing Innowave to its competitors
        canvas_analysis = ctx.data["llm"].complete(
            f"Generate a strategy canvas comparing Innowave's competitive factors with competitors in the market. Highlight areas for value innovation."
        ).text
        return ResponseEvent(query=ev.query, source="Strategy Canvas", response=canvas_analysis)

    @step
    async def four_actions_analysis(self, ctx: Context, ev: BlueOceanEvent) -> ResponseEvent:
        # Analyze Innowave's current strategy and identify factors to eliminate, reduce, raise, and create
        blue_ocean_analysis = ctx.data["llm"].complete(
            f"Analyze Innowave's current strategy and apply the Four Actions Framework (eliminate, reduce, raise, create) to identify Blue Ocean opportunities in the '{ev.query}' domain."
        ).text
        return ResponseEvent(query=ev.query, source="Four Actions Analysis", response=blue_ocean_analysis)

    @step
    async def generate_strategy_recommendations(self, ctx: Context, ev: ResponseEvent) -> StopEvent | None:
        ready = ctx.collect_events(ev, [ResponseEvent] * 2)
        print(f"Collected events: {ready}")

        if not ready or len(ready) < 2:
           print("Not enough responses collected, attempting fallback strategy.")

           # Attempt to generate recommendations with available data
           company_history = ctx.data.get("company_history_response", "No company history available")
           market_analysis = ctx.data.get("market_analysis_response", "No market analysis available")

           fallback_recommendation = ctx.data["llm"].complete(
               f"Create a strategic recommendation based on: "
               f"\n1. Company History: {company_history}"
               f"\n2. Market Analysis: {market_analysis}"
               f"\nNote: Some data might be missing."
           ).text

           return StopEvent(result=fallback_recommendation)

        try:
           company_history = ctx.data.get("company_history_response", "No company history available")
           market_analysis = ctx.data.get("market_analysis_response", "No market analysis available")
           strategy_canvas_response = None
           four_actions_response = None

           for response in ready:
               if response.source == "Strategy Canvas":
                   strategy_canvas_response = response.response
               elif response.source == "Four Actions Analysis":
                   four_actions_response = response.response

           # Generate recommendations with all available data
           strategy_recommendations = ctx.data["llm"].complete(
               f"Create a strategic recommendation based on: "
               f"\n1. Company History: {company_history}"
               f"\n2. Market Analysis: {market_analysis}"
               f"\n3. Strategy Canvas: {strategy_canvas_response or 'No Strategy Canvas available'}"
               f"\n4. Four Actions Analysis: {four_actions_response or 'No Four Actions Analysis available'}"
           ).text
           print("Strategy recommendations generated successfully.")
           return StopEvent(result=str(strategy_recommendations))

        except Exception as e:
           print(f"Error during strategy generation: {e}")
           return StopEvent(result="Error in generating strategy recommendations.")

# Draw the workflow diagram
draw_all_possible_flows(BusinessStrategyWorkflow, filename="business_strategy_workflow.html")

# Set up asyncio for Jupyter
nest_asyncio.apply()

# Run the workflow
async def run_workflow():
    workflow = BusinessStrategyWorkflow(timeout=600, verbose=True)
    result = await workflow.run(query="Identify untapped market segments that InnoWave Inc. can explore to differentiate itself from competitors.", company_id="Innowave")
    return result

# Execute the workflow
result = get_event_loop().run_until_complete(run_workflow())
print(result)

# Display the result in markdown format
from IPython.display import display, HTML
import markdown

display(HTML(markdown.markdown(result)))


business_strategy_workflow.html
Running step analyze_strategy_query
Loading existing index from data/company_history...
Loading existing index from data/market_analysis...
Step analyze_strategy_query produced event CompanyHistoryEvent
Running step company_history_analysis
Company History Analysis produced a response.
Step company_history_analysis produced event MarketAnalysisEvent
Running step market_analysis
Market Analysis produced a response.
Step market_analysis produced event BlueOceanEvent
Running step strategy_canvas
Step strategy_canvas produced event ResponseEvent
Running step four_actions_analysis
Step four_actions_analysis produced event ResponseEvent
Running step generate_strategy_recommendations
Collected events: None
Not enough responses collected, attempting fallback strategy.
Step generate_strategy_recommendations produced event StopEvent
Running step generate_strategy_recommendations
Collected events: [ResponseEvent(query='Identify untapped market segments that InnoWav

In [ ]:
# code for 3. Utilize Default or Cached Data
import os
import asyncio
import nest_asyncio
from asyncio import get_event_loop
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.workflow import step, Context, Workflow, Event, StartEvent, StopEvent
from llama_index.llms.openai import OpenAI
from llama_index.utils.workflow import (
    draw_all_possible_flows,
    draw_most_recent_execution,
)
import subprocess


os.makedirs("data", exist_ok=True)



# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = "Your_api_key_here"

# Define event classes
class StrategyQueryEvent(Event):
    query: str
    company_id: str

class VagueStrategyEvent(Event):
    query: str
    company_id: str

class CompanyHistoryEvent(Event):
    query: str
    company_id: str

class MarketAnalysisEvent(Event):
    query: str
    company_id: str

class BlueOceanEvent(Event):
    query: str
    company_id: str

class ResponseEvent(Event):
    query: str
    source: str
    response: str

# Define the workflow
class BusinessStrategyWorkflow(Workflow):
    def load_or_create_index(self, directory_path, persist_dir):
        full_directory_path = directory_path
        full_persist_dir = os.path.join("data", persist_dir)
        if os.path.exists(full_persist_dir):
            print(f"Loading existing index from {full_persist_dir}...")
            storage_context = StorageContext.from_defaults(persist_dir=full_persist_dir)
            index = load_index_from_storage(storage_context)
        else:
            print(f"Creating new index from {full_directory_path}...")
            documents = SimpleDirectoryReader(full_directory_path).load_data()
            index = VectorStoreIndex.from_documents(documents)
            index.storage_context.persist(persist_dir=full_persist_dir)
        return index

    @step
    async def analyze_strategy_query(self, ctx: Context, ev: StartEvent | StrategyQueryEvent) -> VagueStrategyEvent | CompanyHistoryEvent | MarketAnalysisEvent | BlueOceanEvent:
        if not hasattr(ctx.data, "llm"):
            ctx.data["llm"] = OpenAI(model="gpt-4o-mini", temperature=0.1)
            ctx.data["company_index"] = self.load_or_create_index("data", "company_history")
            ctx.data["market_index"] = self.load_or_create_index("data", "market_analysis")

        query_analysis = ctx.data["llm"].complete(
            f"Analyze this strategy query: '{ev.query}'. Determine if it is clear or vague. If clear, return 'clear'. If vague, return 'vague'."
        ).text
        if query_analysis == "vague":
            return VagueStrategyEvent(query=ev.query, company_id=ev.company_id)
        else:
            return CompanyHistoryEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def refine_strategy_query(self, ctx: Context, ev: VagueStrategyEvent) -> StrategyQueryEvent:
        # Refine the vague strategy query
        refined_query = ctx.data["llm"].complete(
            f"The query '{ev.query}' is too vague for a detailed strategy analysis. Please provide a more specific query."
        ).text
        return StrategyQueryEvent(query=str(refined_query), company_id=ev.company_id)

    @step
    async def company_history_analysis(self, ctx: Context, ev: CompanyHistoryEvent) -> MarketAnalysisEvent:
        # Analyze Innowave's company history
        company_history_analysis = ctx.data["company_index"].as_query_engine().query(
            f"Analyze Innowave's company history and its implications for the strategy related to '{ev.query}'."
        )
        if company_history_analysis:
            print("Company History Analysis produced a response.")
        else:
            print("Company History Analysis did not produce a response.")
        ctx.data["company_history_response"] = company_history_analysis
        return MarketAnalysisEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def market_analysis(self, ctx: Context, ev: MarketAnalysisEvent) -> BlueOceanEvent:
        # Analyze market conditions
        market_analysis = ctx.data["market_index"].as_query_engine().query(
            f"Analyze the market conditions and competitive landscape for Innowave's strategy in '{ev.query}'."
        )
        if market_analysis:
            print("Market Analysis produced a response.")
        else:
            print("Market Analysis did not produce a response.")
        ctx.data["market_analysis_response"] = market_analysis
        return BlueOceanEvent(query=ev.query, company_id=ev.company_id)

    @step
    async def strategy_canvas(self, ctx: Context, ev: BlueOceanEvent) -> ResponseEvent:
        # Generate a strategy canvas comparing Innowave to its competitors
        canvas_analysis = ctx.data["llm"].complete(
            f"Generate a strategy canvas comparing Innowave's competitive factors with competitors in the market. Highlight areas for value innovation."
        ).text
        return ResponseEvent(query=ev.query, source="Strategy Canvas", response=canvas_analysis)

    @step
    async def four_actions_analysis(self, ctx: Context, ev: BlueOceanEvent) -> ResponseEvent:
        # Analyze Innowave's current strategy and identify factors to eliminate, reduce, raise, and create
        blue_ocean_analysis = ctx.data["llm"].complete(
            f"Analyze Innowave's current strategy and apply the Four Actions Framework (eliminate, reduce, raise, create) to identify Blue Ocean opportunities in the '{ev.query}' domain."
        ).text
        return ResponseEvent(query=ev.query, source="Four Actions Analysis", response=blue_ocean_analysis)

    @step
    async def generate_strategy_recommendations(self, ctx: Context, ev: ResponseEvent) -> StopEvent | None:
        ready = ctx.collect_events(ev, [ResponseEvent] * 2)
        print(f"Collected events: {ready}")

        if not ready or len(ready) < 2:
           print("Not enough responses collected, using default data.")

           # Use default or cached data
           company_history = ctx.data.get("company_history_response", "Default Company History")
           market_analysis = ctx.data.get("market_analysis_response", "Default Market Analysis")

           strategy_recommendations = ctx.data["llm"].complete(
               f"Generate strategic recommendations using default data."
               f"\nCompany History: {company_history}"
               f"\nMarket Analysis: {market_analysis}"
           ).text

           return StopEvent(result=str(strategy_recommendations))

        try:
           company_history = ctx.data.get("company_history_response", "No company history available")
           market_analysis = ctx.data.get("market_analysis_response", "No market analysis available")
           strategy_canvas_response = None
           four_actions_response = None

           for response in ready:
               if response.source == "Strategy Canvas":
                   strategy_canvas_response = response.response
               elif response.source == "Four Actions Analysis":
                   four_actions_response = response.response

           # Generate recommendations with all available data
           strategy_recommendations = ctx.data["llm"].complete(
               f"Create a strategic recommendation based on: "
               f"\n1. Company History: {company_history}"
               f"\n2. Market Analysis: {market_analysis}"
               f"\n3. Strategy Canvas: {strategy_canvas_response or 'No Strategy Canvas available'}"
               f"\n4. Four Actions Analysis: {four_actions_response or 'No Four Actions Analysis available'}"
           ).text
           print("Strategy recommendations generated successfully.")
           return StopEvent(result=str(strategy_recommendations))

        except Exception as e:
           print(f"Error during strategy generation: {e}")
           return StopEvent(result="Error in generating strategy recommendations.")

# Draw the workflow diagram
draw_all_possible_flows(BusinessStrategyWorkflow, filename="business_strategy_workflow.html")

# Set up asyncio for Jupyter
nest_asyncio.apply()

# Run the workflow
async def run_workflow():
    workflow = BusinessStrategyWorkflow(timeout=600, verbose=True)
    result = await workflow.run(query="Identify untapped market segments that InnoWave Inc. can explore to differentiate itself from competitors.", company_id="Innowave")
    return result

# Execute the workflow

result = get_event_loop().run_until_complete(run_workflow())
draw_most_recent_execution(workflow, filename="consulting_flow_recent.html")
print(result)

# Display the result in markdown format
from IPython.display import display, HTML
import markdown

display(HTML(markdown.markdown(result)))


ModuleNotFoundError: No module named 'llama_index'